In [1]:
import vertexai
from vertexai.preview import reasoning_engines

PROJECT_ID = "aiuda-ffc77"

vertexai.init(
    project=PROJECT_ID,
    location="us-central1",
    staging_bucket="gs://aiuda-docs",
)

In [2]:
model = "gemini-1.0-pro"

In [3]:
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai

In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

# TODO(developer): Update and un-comment below lines
# project_id = "PROJECT_ID"

# vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.5-flash-001")

prompt = """
Can you transcribe this interview, in the format of timecode, speaker, caption.
Use speaker A, speaker B, etc. to identify speakers.
"""

audio_file_uri = "gs://cloud-samples-data/generative-ai/audio/pixel.mp3"
audio_file = Part.from_uri(audio_file_uri, mime_type="audio/mpeg")

contents = [audio_file, prompt]

response = model.generate_content(contents)
print(response.text)

0:00 Speaker A Your devices are getting better over time and so, we think about it across the entire portfolio from phones to watch to buds to tablet. We get really excited about how we can tell a joint narrative across everything.
0:18 Speaker B Welcome to the Made by Google podcast, where we meet the people who work on the Google products you love. Here's your host, Rasheed Finch. Today, we're talking to Aisha Sharif and DeCarlos Love. They're both product managers for various Pixel devices and work on something that all the Pixel owners love, the Pixel feature drops. This is the Made by Google Podcast. Aisha, which feature on your Pixel phone has been most transformative in your own life?
0:54 Speaker C So many features. I am a singer, so I actually think recorded transcription has been incredible. Because before I would record songs, I just like freestyle them, record them, type them up, but now with transcription, it works so well, even deciphering lyrics that are jumbled, I think

In [4]:
response = model.generate_content([response.text, "Translate to Spanish"])
print(response.text)

## Traducción al español

0:00 Orador A Tus dispositivos están mejorando con el tiempo y por eso, pensamos en ello en todo el portafolio, desde teléfonos hasta relojes, auriculares y tabletas. Nos emociona mucho cómo podemos contar una narrativa conjunta en todo.
0:18 Orador B Bienvenidos al podcast de Made by Google, donde conocemos a las personas que trabajan en los productos de Google que te encantan. Aquí está tu presentador, Rasheed Finch. Hoy, hablamos con Aisha Sharif y DeCarlos Love. Ambos son gerentes de producto para varios dispositivos Pixel y trabajan en algo que todos los propietarios de Pixel adoran: las caídas de funciones de Pixel. Este es el podcast de Made by Google. Aisha, ¿qué función en tu teléfono Pixel ha sido más transformadora en tu propia vida?
0:54 Orador C Tantas funciones. Soy cantante, así que creo que la transcripción grabada ha sido increíble. Porque antes grababa canciones, solo las improvisaba, las grababa, las escribía, pero ahora con la transcripción

In [4]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
    # HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

In [5]:
model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens.
    "top_k": 40,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    # "safety_settings": safety_settings,
}

In [6]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain
    exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code).
            Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code).
            Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate.
            Defaults to "latest" for the most recent exchange rate data.
            Can be specified in YYYY-MM-DD format for historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
            Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24",
                "rates": {"EUR": 0.95534}}
    """
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [7]:
# Test the function

get_exchange_rate(currency_from="USD", currency_to="SEK")

{'amount': 1.0, 'base': 'USD', 'date': '2024-07-01', 'rates': {'SEK': 10.5793}}

In [8]:
agent = reasoning_engines.LangchainAgent(
    model=model,  # Required.
    tools=[get_exchange_rate],  # Optional.
    model_kwargs=model_kwargs,  # Optional.
)

In [9]:
response = agent.query(
    input="What is the exchange rate from US dollars to Brasilian Real currency?"
)

In [10]:
response

{'input': 'What is the exchange rate from US dollars to Brasilian Real currency?',
 'output': 'The exchange rate from US dollars to Brazilian Real as of today, 2024-07-01, is 1 USD = 5.5855 BRL. \n'}

# Deploy de App

In [11]:
DISPLAY_NAME = "Demo Langchain Application"

remote_app = reasoning_engines.ReasoningEngine.create(
    reasoning_engines.LangchainAgent(
        model=model,
        tools=[get_exchange_rate],
        model_kwargs=model_kwargs,
    ),
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]",
    ],
    display_name=DISPLAY_NAME,
)
remote_app

Using bucket aiuda-docs
Writing to gs://aiuda-docs/reasoning_engine/reasoning_engine.pkl
Writing to gs://aiuda-docs/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://aiuda-docs/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352/operations/4227073311978291200
ReasoningEngine created. Resource name: projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352')


resource name: projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352

In [12]:
#!pip install google-cloud-aiplatform[reasoningengine]

In [13]:
remote_app.resource_name

'projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352'

In [15]:
response = remote_app.query(
    input="What is the exchange rate from US dollars to Swedish currency?"
)

In [17]:
response["output"]

'The exchange rate from US dollars to Swedish krona (SEK) is 1 USD = 10.5793 SEK as of July 1, 2024.'

# Manage the deployed application

In [18]:
reasoning_engines.ReasoningEngine.list()

 resource name: projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352,
 resource name: projects/286295157330/locations/us-central1/reasoningEngines/5808552803773186048]

In [19]:
reasoning_engines.ReasoningEngine.list(
    filter='display_name="Demo Langchain Application"'
)

 resource name: projects/286295157330/locations/us-central1/reasoningEngines/5870899511114596352]